In [2]:
import os
import pandas as pd
from docx import Document
from sentence_transformers import SentenceTransformer, util
import seaborn as sns
import matplotlib.pyplot as plt

# 1. خواندن فایل‌های DOCX از پوشه
def read_docx_files(folder_path):
    docx_files = [f for f in os.listdir(folder_path) if f.endswith('.docx')]
    doc_texts = {}
    
    for file in docx_files:
        doc = Document(os.path.join(folder_path, file))
        full_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip() != ""])
        doc_texts[file] = full_text
    
    return doc_texts

# 2. بارگذاری داده‌های اکسل با ساختار جدید
df = pd.read_excel('C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/similar_topics_above_50.xlsx')
df = df[['Topic_1', 'Course_1', 'Topic_2', 'Course_2']]

# 3. مدل پیش‌فرض برای مشابه‌سازی
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 4. خواندن فایل‌های DOCX از پوشه مشخص
folder_path = 'C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/Similarity'
docx_texts = read_docx_files(folder_path)

# 5. ایجاد جدول شباهت‌ها از مقایسه محتوای فایل‌ها
similar_pairs = []
threshold = 0.7

for _, row in df.iterrows():
    topic1, course1 = row['Topic_1'], row['Course_1']
    topic2, course2 = row['Topic_2'], row['Course_2']
    
    # پیدا کردن فایل‌های مرتبط با این دو عنوان
    files_topic1 = [f for f in docx_texts.keys() if topic1.lower() in f.lower()]
    files_topic2 = [f for f in docx_texts.keys() if topic2.lower() in f.lower()]
    
    if files_topic1 and files_topic2:
        file1 = files_topic1[0]
        file2 = files_topic2[0]
        
        text1 = docx_texts[file1]
        text2 = docx_texts[file2]
        
        # محاسبه شباهت محتوای دو متن
        content_similarity_score = util.pytorch_cos_sim(
            model.encode([text1, text2], convert_to_tensor=True)[0],
            model.encode([text1, text2], convert_to_tensor=True)[1]
        ).cpu().numpy()[0][0]
        
        # چاپ شباهت و بررسی اینکه آیا از آستانه عبور می‌کند
        print(f"Similarity score between {topic1} and {topic2}: {content_similarity_score}")
        
        if content_similarity_score > threshold:
            # افزودن داده‌ها به لیست مشابهت‌ها
            similar_pairs.append({
                'Topic 1': topic1,
                'Course 1': course1,
                'Topic 2': topic2,
                'Course 2': course2,
                'Similarity': round(content_similarity_score, 3),
                'File 1': file1,
                'File 2': file2,
                'Content Similarity': round(content_similarity_score, 3),
                'Sentence 1': None,
                'Sentence 2': None,
                'Sentence Similarity': None
            })

# 6. بررسی اینکه آیا داده‌ها به `similar_pairs` اضافه شده‌اند
if len(similar_pairs) == 0:
    print("No similar pairs found. Check if the similarity scores are higher than the threshold.")

# 7. ایجاد DataFrame برای نتایج مشابهت
similar_df = pd.DataFrame(similar_pairs)

# 8. چاپ نام ستون‌ها و سطرهای اولیه برای بررسی
print("Columns in DataFrame:", similar_df.columns)
print("First 5 rows of DataFrame:\n", similar_df.head())

# 9. نمایش نتایج در کنسول
if not similar_df.empty:
    print(similar_df[['Topic 1', 'Course 1', 'Topic 2', 'Course 2', 'Similarity', 'File 1', 'File 2', 'Content Similarity']])
else:
    print("No data to display.")

# 10. ذخیره نتایج در فایل Excel
output_path = "C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/docx_comparison_with_content_similarity.xlsx"
similar_df.to_excel(output_path, index=False)

# 11. رسم نمودار Heatmap برای شباهت‌ها
if not similar_df.empty:
    plt.figure(figsize=(12, 10))
    sns.heatmap(similar_df.pivot_table(index='Topic 1', columns='Topic 2', values='Similarity'), cmap="YlGnBu", annot=True)
    plt.title("Document Similarity Heatmap")
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig("docx_similarity_heatmap.png")
    plt.show()


No similar pairs found. Check if the similarity scores are higher than the threshold.
Columns in DataFrame: RangeIndex(start=0, stop=0, step=1)
First 5 rows of DataFrame:
 Empty DataFrame
Columns: []
Index: []
No data to display.


In [4]:
import os
import pandas as pd
from docx import Document
from sentence_transformers import SentenceTransformer, util
import seaborn as sns
import matplotlib.pyplot as plt

# 1. خواندن فایل‌های DOCX از پوشه
def read_docx_files(folder_path):
    docx_files = [f for f in os.listdir(folder_path) if f.endswith('.docx')]
    doc_texts = {}
    
    for file in docx_files:
        doc = Document(os.path.join(folder_path, file))
        full_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip() != ""])
        doc_texts[file] = full_text
    
    return doc_texts

# 2. بارگذاری داده‌های اکسل با ساختار جدید
df = pd.read_excel('C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/similar_topics_above_50.xlsx')
df = df[['Topic_1', 'Course_1', 'Topic_2', 'Course_2']]

# 3. مدل پیش‌فرض برای مشابه‌سازی
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# 4. خواندن فایل‌های DOCX از پوشه مشخص
folder_path = 'C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/Similarity'
docx_texts = read_docx_files(folder_path)

# 5. ایجاد جدول شباهت‌ها از مقایسه محتوای فایل‌ها
similar_pairs = []
threshold = 0.5  # کاهش آستانه شباهت

for _, row in df.iterrows():
    topic1, course1 = row['Topic_1'], row['Course_1']
    topic2, course2 = row['Topic_2'], row['Course_2']
    
    # پیدا کردن فایل‌های مرتبط با این دو عنوان
    files_topic1 = [f for f in docx_texts.keys() if topic1.lower() in f.lower()]
    files_topic2 = [f for f in docx_texts.keys() if topic2.lower() in f.lower()]
    
    if files_topic1 and files_topic2:
        file1 = files_topic1[0]
        file2 = files_topic2[0]
        
        text1 = docx_texts[file1]
        text2 = docx_texts[file2]
        
        # محاسبه شباهت محتوای دو متن
        content_similarity_score = util.pytorch_cos_sim(
            model.encode([text1, text2], convert_to_tensor=True)[0],
            model.encode([text1, text2], convert_to_tensor=True)[1]
        ).cpu().numpy()[0][0]
        
        # چاپ شباهت و بررسی اینکه آیا از آستانه عبور می‌کند
        print(f"Similarity score between {topic1} and {topic2}: {content_similarity_score}")
        
        if content_similarity_score > threshold:
            # افزودن داده‌ها به لیست مشابهت‌ها
            similar_pairs.append({
                'Topic 1': topic1,
                'Course 1': course1,
                'Topic 2': topic2,
                'Course 2': course2,
                'Similarity': round(content_similarity_score, 3),
                'File 1': file1,
                'File 2': file2,
                'Content Similarity': round(content_similarity_score, 3),
                'Sentence 1': None,
                'Sentence 2': None,
                'Sentence Similarity': None
            })

# 6. بررسی اینکه آیا داده‌ها به `similar_pairs` اضافه شده‌اند
if len(similar_pairs) == 0:
    print("No similar pairs found. Check if the similarity scores are higher than the threshold.")

# 7. ایجاد DataFrame برای نتایج مشابهت
similar_df = pd.DataFrame(similar_pairs)

# 8. چاپ نام ستون‌ها و سطرهای اولیه برای بررسی
print("Columns in DataFrame:", similar_df.columns)
print("First 5 rows of DataFrame:\n", similar_df.head())

# 9. نمایش نتایج در کنسول
if not similar_df.empty:
    print(similar_df[['Topic 1', 'Course 1', 'Topic 2', 'Course 2', 'Similarity', 'File 1', 'File 2', 'Content Similarity']])
else:
    print("No data to display.")

# 10. ذخیره نتایج در فایل Excel
output_path = "C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/docx_comparison_with_content_similarity.xlsx"
similar_df.to_excel(output_path, index=False)

# 11. رسم نمودار Heatmap برای شباهت‌ها
if not similar_df.empty:
    plt.figure(figsize=(12, 10))
    sns.heatmap(similar_df.pivot_table(index='Topic 1', columns='Topic 2', values='Similarity'), cmap="YlGnBu", annot=True)
    plt.title("Document Similarity Heatmap")
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig("C:/Users/Nastaran/Desktop/UniversityConstructor/semester4/Final 08042025/Data/docx_similarity_heatmap.png")
    plt.show()


No similar pairs found. Check if the similarity scores are higher than the threshold.
Columns in DataFrame: RangeIndex(start=0, stop=0, step=1)
First 5 rows of DataFrame:
 Empty DataFrame
Columns: []
Index: []
No data to display.
